In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

class SentimentAnalysisModel:
    def __init__(self, model_name="bert-base-uncased", num_labels=2):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    def preprocess(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        return inputs

    def predict(self, text):
        inputs = self.preprocess(text)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)
        return predicted_class.item()

    def train(self, train_dataloader, epochs=5, learning_rate=2e-5):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.model.train()
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=learning_rate)

        for epoch in range(epochs):
            for batch in train_dataloader:
                input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                loss, logits = outputs
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

    def evaluate(self, val_dataloader):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy

In [2]:
import torch
import random
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import matplotlib.pyplot as plt

class SentimentAnalysisModel:
    def __init__(self, model_name="distilbert-base-uncased", num_labels=2):
        self.tokenizer = DistilBertTokenizer.from_pretrained(model_name)
        self.model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    def preprocess(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        return inputs

    def predict(self, text):
        inputs = self.preprocess(text)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)
        return predicted_class.item()

    def train(self, train_dataloader, epochs=5, learning_rate=2e-5):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.model.train()
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=learning_rate)

        for epoch in range(epochs):
            for batch in train_dataloader:
                input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                loss, logits = outputs
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

    def evaluate(self, val_dataloader):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy
    
    def visualize_data(self, data, labels):
        # Assuming data is a list of text strings and labels is a list of 0s and 1s
        positive_examples = [text for text, label in zip(data, labels) if label == 1]
        negative_examples = [text for text, label in zip(data, labels) if label == 0]

        # Visualize a random sample of positive and negative examples
        num_examples = 5
        fig, axs = plt.subplots(2, num_examples, figsize=(15, 5))
        for i in range(num_examples):
            axs[0, i].text(0.5, 0.5, positive_examples[i], ha='center', va='center')
            axs[0, i].axis('off')
            axs[1, i].text(0.5, 0.5, negative_examples[i], ha='center', va='center')
            axs[1, i].axis('off')

        plt.show()

def inject_random_bit_flip(model):
    # Get a random layer and parameter index
    layer_index = random.randint(0, len(list(model.parameters())) - 1)
    parameter_index = random.randint(0, len(model.parameters()[layer_index]) - 1)

    # Get the parameter tensor
    param = model.parameters()[layer_index][parameter_index]

    # Get a random bit position
    bit_position = random.randint(0, param.numel() * 8 - 1)

    # Flip the bit at the specified position
    param_data = param.data.numpy()
    byte_index = bit_position // 8
    bit_mask = 1 << (bit_position % 8)
    param_data[byte_index] ^= bit_mask
    param.data = torch.tensor(param_data, dtype=torch.float32)

    return layer_index, parameter_index, bit_position

def run_experiment(model, val_dataloader, num_flips):
    original_accuracy = model.evaluate(val_dataloader)
    print(f"Original accuracy: {original_accuracy:.2f}%")

    flip_results = []
    for i in range(num_flips):
        layer_index, parameter_index, bit_position = inject_random_bit_flip(model)
        new_accuracy = model.evaluate(val_dataloader)
        flip_results.append((layer_index, parameter_index, bit_position, new_accuracy))
        print(f"Flip {i+1}: Layer {layer_index}, Parameter {parameter_index}, Bit {bit_position}, New accuracy: {new_accuracy:.2f}%")

    return flip_results


In [ ]:
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize the text using the loaded tokenizer and convert it to PyTorch tensors
        encoded_text = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        return encoded_text, label

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.2)

# Create the datasets and dataloaders
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
